Continuing work from preprocessing1a, just because that notebook was running too slowly: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/preprocessing1a_choose_users.ipynb

Previously, I selected a random subset of users and created rows for non-orders. Here, I'll need to start by cleaning up all those rows before building the 'past_orders' column. 

In [1]:
import pandas as pd
import numpy as np
import os
from library.sb_utils import save_file

import random

In [2]:
from IPython.display import Audio
sound_file = './alert.wav'

In [3]:
# import users_incorrect_deets from previous notebook
df = pd.read_csv('../data/processed/users_incorrect_deets.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1316889 entries, 0 to 1316888
Data columns (total 11 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   order_id                1316889 non-null  object
 1   user_id                 1316889 non-null  int64 
 2   order_by_user_sequence  1316889 non-null  int64 
 3   order_dow               1316889 non-null  object
 4   order_hour_of_day       1316889 non-null  object
 5   days_since_prior_order  1316889 non-null  object
 6   add_to_cart_sequence    1316889 non-null  object
 7   reordered               1316889 non-null  object
 8   product_name            1316889 non-null  object
 9   aisle_name              1316889 non-null  object
 10  dept_name               1316889 non-null  object
dtypes: int64(2), object(9)
memory usage: 110.5+ MB


In [11]:
# The new rows were created without updating orde_id, order_dow, order_hour_of_day, 
# days_since_prior_order. Fix this.

users = list(df['user_id'].unique())

for user in users:
    this_users_rows = df.loc[df['user_id']==user, :].copy()
    
    # Iterate over each order for this user
    for order in range(2, 101):
        
        this_order = this_users_rows.loc[this_users_rows[
            'order_by_user_sequence']==order, :].copy()
        
        # Get the values columns should have for this order. Can just get the order details 
        # from the first item placed in the cart. Need to ignore index errors that come up 
        # when a particular user doesn't actually have as many orders as I'm interating through
        # in order to just move on to the next user. 
        try:
            correct_order_id = this_order[this_order['add_to_cart_sequence']==1][
                'order_id'].values[0]
            correct_order_dow = this_order[this_order['add_to_cart_sequence']==1][
            'order_dow'].values[0]
            correct_order_hour_of_day = this_order[this_order['add_to_cart_sequence']==1][
                'order_hour_of_day'].values[0]
            correct_days_since_prior_order = this_order[this_order['add_to_cart_sequence']==1][
                'days_since_prior_order'].values[0]
        except IndexError:
            pass
        
        # Change values in original df, just rows for this order. So that all rows, including
        # those I created and filled with 'x', have the correct order details
        try:
            df.loc[(df['user_id']==user) 
                   &(df['order_by_user_sequence']==order),'order_id'] = correct_order_id
            df.loc[(df['user_id']==user)
                   &(df['order_by_user_sequence']==order),'order_dow'] = correct_order_dow
            df.loc[(df['user_id']==user)
                   &(df['order_by_user_sequence']==order),'order_hour_of_day'
                  ] = correct_order_hour_of_day
            df.loc[(df['user_id']==user)&
                   (df['order_by_user_sequence']==order),'days_since_prior_order'
                  ] = correct_days_since_prior_order
        except NameError:
            pass

In [12]:
# Check to see if the change was made correctly.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1316889 entries, 0 to 1316888
Data columns (total 11 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   order_id                1316889 non-null  object
 1   user_id                 1316889 non-null  int64 
 2   order_by_user_sequence  1316889 non-null  int64 
 3   order_dow               1316889 non-null  object
 4   order_hour_of_day       1316889 non-null  object
 5   days_since_prior_order  1316889 non-null  object
 6   add_to_cart_sequence    1316889 non-null  object
 7   reordered               1316889 non-null  object
 8   product_name            1316889 non-null  object
 9   aisle_name              1316889 non-null  object
 10  dept_name               1316889 non-null  object
dtypes: int64(2), object(9)
memory usage: 110.5+ MB


In [14]:
df['user_id'].unique()

array([  1697,   2396,   3062,   4015,   7175,   7918,   8198,  10110,
        10261,  10371,  13206,  13618,  15391,  15652,  15878,  16853,
        17495,  18575,  18688,  20533,  21516,  23944,  24148,  25523,
        26130,  26270,  26445,  29270,  29963,  30393,  34517,  35052,
        35830,  39908,  40299,  40628,  42087,  43409,  43556,  43858,
        44279,  44494,  44848,  45771,  48626,  48954,  49329,  50114,
        50905,  52003,  52621,  54136,  54312,  55493,  57208,  58208,
        58440,  58927,  60140,  61203,  61543,  61727,  64200,  66168,
        67143,  70202,  72938,  74231,  74294,  74453,  76588,  78486,
        80642,  82114,  83098,  83368,  83994,  86299,  86437,  86520,
        86952,  87052,  88620,  89018,  89187,  90508,  91497,  93036,
        93698,  94945,  95481,  98163,  99480, 100545, 101338, 102326,
       104505, 105991, 106162, 106644, 107036, 108138, 108181, 112535,
       113338, 114307, 115201, 117541, 117760, 118114, 118260, 119297,
      

In [17]:
df[df['user_id']==196361]

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name
35073,650934,196361,1,0,18,-1.0,19,0,Air Chilled Organic Boneless Skinless Chicken ...,poultry counter,meat seafood
35074,650934,196361,1,0,18,-1.0,1,0,Bag of Organic Bananas,fresh fruits,produce
35075,650934,196361,1,0,18,-1.0,20,0,Organic Red Onion,fresh vegetables,produce
35076,650934,196361,1,0,18,-1.0,5,0,1% Lowfat Milk,milk,dairy eggs
35077,650934,196361,1,0,18,-1.0,22,0,Mango Chunks,frozen produce,frozen
...,...,...,...,...,...,...,...,...,...,...,...
666457,x,196361,100,x,x,x,x,x,Organic Mexican Blend Finely Shredded Cheese,x,x
666458,x,196361,100,x,x,x,x,x,Heart to Heart Honey Toasted Oat Cereal,x,x
666459,x,196361,100,x,x,x,x,x,Cuties Non Dairy Vanilla Frozen Dessert Sandwi...,x,x
666460,x,196361,100,x,x,x,x,x,Garlic Powder,x,x


In [ ]:
df['user_id'].unique()

In [ ]:
df[(df['user_id']==10871) & df['order_by_user_sequence'].isin([
    2,3])].sort_values('order_by_user_sequence')

Each order now has the same order details (order_id, day, hour, days_since_prior) in each row of the order, correctly matching items actually purchased in that order.

Now I need to use the dictionaries I created earlier to fill in the aisle_name and dept_name based on the product_name. 

Up next is preparing to create a column that indicates how many times any given item has been ordered in the past. 

My intuition is that cross-row calculations don't count as leakage and negatively impact modeling IF I'm adding data that has exclusively to do with past orders. If this logic turns out to be inappropriate, I can just come back and split the set into separate users (or into certain orders per user) and re-run any subsequent code. 

In [ ]:
# I might want a dictionary of which products were reordered in each order, by each user. 

prod_reorder_by_user_per_order = {}

for user in users:
    products_reordered_each_order = {}
    for order in range(2,101):
        items = set(df[(df['user_id']==user) & (df[
            'order_by_user_sequence']==order) & (df[
            'reordered']==1)]['product_name'])
        if len(items)>0:
            products_reordered_each_order[order]=items
    prod_reorder_by_user_per_order[user]=products_reordered_each_order
    
prod_reorder_by_user_per_order

Create the column itself, starting as just a copy of 'reordered'. 

For each user's first order, this column will already be true because reordered=0 and past_orders=0 since there were no past orders. For their second order, it will be partly true. For order 2, if rordered=1, then it's because the item was ordered once in the past (in the first order). For order 2, if add_to_cart_sequence=0, the past_order value should actually be 1 because that indicates non-orders for order 2 of items previously ordered. The only items in order 2 that should have a past_order value of 0 should be items being ordered for the first time in order 2. 

In [ ]:
df['past_orders'] = df.loc[:,'reordered'].copy()

In [ ]:
# To start with, just work with 2 users, to more quickly see if the code works

all_users = set(df['user_id'].unique())
practice_users = random.sample(list(all_users), 2)
df0 = df.loc[df['user_id'].isin(practice_users), :].copy()
df0.info()

In [ ]:
# Do the work here

In [ ]:
# Check to see if this worked. 

In [ ]:
# Engineer columns for product keywords

In [ ]:
# Change format of dow, hour columns

In [ ]:
# Can't do the rest of the normalization until I have a train/test split and pipeline for it
# Save work done so far as new csv file and pick up in next notebook

Move to new notebook for encoding & standardizing remaining features